In [ ]:
import pandas as pd


data = pd.DataFrame(columns=['date','profit_rate','account_money','year'])

data.loc[data.shape[0]]=['2004-05-13',0,100000,2004]
data.loc[data.shape[0]]=['2004-07-30',-6.84932,97070,2004]
data.loc[data.shape[0]]=['2004-09-27',-6.42202,94403,2004]
data.loc[data.shape[0]]=['2004-12-14',-4.58716,92548,2004]

data.loc[data.shape[0]]=['2005-01-31',-11.5044,87985,2005]
data.loc[data.shape[0]]=['2005-01-31',-11.5044,87985,2005]
data.loc[data.shape[0]]=['2005-03-14',-5.40541,85951,2005]
data.loc[data.shape[0]]=['2006-06-04',-7.40541,81951,2006]


data

In [ ]:
import pandas as pd
dir = '/Users/xinwang/stock/data/day/'
file_name=  'day_report_df.csv'

sections = pd.read_csv(dir+file_name)

sections['ampl'].shape

In [ ]:
bins = [-100,0,20,40,60,70,90,120,150,500]
cuts = pd.cut(sections['ampl'], bins)

def get_status(group):
    return {
        'count':group.count()*100.0/1942
    }
    

bins_count = sections['ampl'].groupby(cuts).apply(get_status).unstack()
bins_count

In [ ]:
sections[(sections['ampl']<=10) & (sections['ampl']>0)].shape

In [ ]:
114*100.0/sections.shape[0]

In [ ]:
sections.shape[0]

In [ ]:
from datetime import datetime
datetime.now().strftime('%Y-%m-%d_%H:%M')

In [4]:
import pandas as pd
import datetime as dt

day_dir='/Users/xinwang/stock/data/day/'
file_name='D_SZ#300168.txt'

k_columns = ['date','open','high','low','close','volume','amount']

stock_data = pd.read_csv(day_dir+file_name,header=None, names=k_columns)

period_type = 'W'
stock_data['date'] = pd.to_datetime(stock_data['date'])

stock_data.set_index('date',inplace=True)

w_data = stock_data.resample(period_type).last()

w_data['open'] = stock_data['open'].resample(period_type).first()
w_data['high'] = stock_data['high'].resample(period_type).max()
w_data['low'] = stock_data['low'].resample(period_type).min()
w_data['volume'] = stock_data['volume'].resample(period_type).sum()
w_data['amount'] = stock_data['amount'].resample(period_type).sum()

w_data.reset_index(inplace=True)
w_data['date'] = w_data['date'].apply(lambda x: x + dt.timedelta(days = -2))
w_data['datetime'] = w_data['date']
w_data['date'] = w_data['date'].apply(lambda x: x.strftime('%Y-%m-%d'))

w_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 422 entries, 0 to 421
Data columns (total 8 columns):
date        422 non-null object
open        373 non-null float64
high        373 non-null float64
low         373 non-null float64
close       373 non-null float64
volume      422 non-null int64
amount      422 non-null float64
datetime    422 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(5), int64(1), object(1)
memory usage: 26.5+ KB


In [11]:
import pandas as pd
import datetime as dt

report_dir='/Users/xinwang/stock/data/report/'
report_columns = ['symbol','s_date','e_date','ampl','ma_per','periods']

day_report_df = pd.read_csv(report_dir+'day_report_df.csv')
week_report_df = pd.read_csv(report_dir+'week_report_df.csv')



In [27]:
import pandas as pd
import numpy as np

day_dir='/Users/xinwang/stock/data/day/'
k_columns = ['date','open','high','low','close','volume','amount']

data = pd.read_csv(day_dir+'D_SZ#002386.txt',header=None, names=k_columns)

def previous_series(cur_date,latest_n):
    cur_index = data[data['date']==cur_date].index.values[0]
    start_index = cur_index.astype(int)-latest_n+1

    latest_part = data[start_index:cur_index+1]

    return latest_part

def ma(cur_date,periods):
    periods_part = previous_series(cur_date, periods)['close']
    return round(periods_part.mean(),2)

def std(cur_date, periods):
    periods_part = previous_series(cur_date, periods)['close']
    return np.std(periods_part)


#     {参数 N: 2，250，20 }
# BOLL:MA(CLOSE,M); UB:BOLL+2*STD(CLOSE,M); LB:BOLL-2*STD(CLOSE,M);
def boll(cur_date):
    M=20
    price = data[data['date']==cur_date]['close'].values[0]

    boll = ma(cur_date, M)
    up_boll = boll + 2 * std(cur_date,M)
    low_boll = boll - 2 * std(cur_date,M)
    print(cur_date+ ' up boll:'+str(up_boll) +' low boll:'+str(low_boll))
    return boll
        

def is_on_boll(cur_date):
    boll_value = boll(cur_date)
    
    price = data[data['date']==cur_date]['close'].values[0]
    
    if(price>boll_value):
        return True
    else:
        return False
        
for date in ['2010-08-12','2010-08-13','2010-08-16','2010-08-17','2010-08-18',
            '2010-08-19','2010-08-20','2010-08-23']:
    print(date +' '+ str(is_on_boll(date)))
    print()


2010-08-12 up boll:11.38652704066211 low boll:9.95347295933789
2010-08-12 False

2010-08-13 up boll:11.350789980896971 low boll:10.08921001910303
2010-08-13 True

2010-08-16 up boll:11.324910612398103 low boll:10.195089387601897
2010-08-16 True

2010-08-17 up boll:11.3309566837323 low boll:10.269043316267702
2010-08-17 True

2010-08-18 up boll:11.281265226366186 low boll:10.398734773633814
2010-08-18 True

2010-08-19 up boll:11.239136464314056 low boll:10.520863535685946
2010-08-19 True

2010-08-20 up boll:11.25440083246528 low boll:10.485599167534719
2010-08-20 False

2010-08-23 up boll:11.28085960590429 low boll:10.41914039409571
2010-08-23 False

